In [1]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [109]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [124]:
query = \
"""
select * from icu_features;
"""
res = pd.read_sql_query(query, con)
res.head()

,icustay_id,hadm_id,subject_id,age,gender,height,weight,ethnicity,insurance,filter_vaso,...,pc_bronch,pc_cath,pc_echo,pc_pressor,pc_rhc,pc_thora,pc_vent,passed_filters,use_record,bmi
0,200001,152234,55973,22290 days 19:06:12,F,167.851667,27.669135,ASIAN - ASIAN INDIAN,Medicare,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,9.820741
1,200003,163557,27513,17625 days 19:50:04,M,177.800000,78.224998,WHITE,Private,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,24.744692
2,200006,189514,10950,19736 days 11:28:14,M,165.100000,82.400002,OTHER,Medicaid,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.229647
3,200007,129310,20707,15818 days 10:03:37,M,177.800000,126.000000,WHITE,Private,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.857223
4,200009,129607,29904,17353 days 10:34:32,F,160.020000,85.833331,WHITE,Private,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,33.520264


In [125]:
(~res['ea_key'].isnull()).value_counts()

False    41780
True     19752
Name: ea_key, dtype: int64

In [126]:
(~res['ea_lv_systolic'].isnull()).value_counts()

False    45612
True     15920
Name: ea_lv_systolic, dtype: int64

In [14]:
res['use_record'].value_counts(dropna = False)

NaN     52212
 1.0     9320
Name: use_record, dtype: int64

In [128]:
res_ = res.loc[res['use_record'] == 1]
res_.shape

(9320, 152)

In [136]:
res_['pc_toexclude'].value_counts(dropna = False)

 0.0    7338
NaN     1982
Name: pc_toexclude, dtype: int64

In [129]:
(~res_['ea_key'].isnull()).value_counts()

False    5634
True     3686
Name: ea_key, dtype: int64

In [38]:
inpatient_sids = res_['subject_id'].unique()

In [145]:
res_['outtime'] - res_['intime']

1        5 days 21:19:14
16       0 days 09:08:59
20       6 days 03:43:57
22      13 days 21:03:01
25       2 days 23:50:23
34       0 days 17:19:21
46      13 days 17:02:57
49       2 days 15:16:43
51       0 days 19:58:51
54       2 days 01:41:57
60       4 days 06:20:20
68       8 days 21:09:49
78       0 days 22:11:56
85       4 days 18:50:05
86       1 days 04:50:55
88       8 days 05:29:19
93       3 days 08:17:59
99      26 days 18:46:34
133      0 days 15:42:43
138      7 days 22:41:45
146     38 days 03:52:16
150      2 days 19:36:32
153      5 days 06:56:13
155      1 days 05:00:20
157      5 days 03:38:31
163     13 days 11:23:43
167      3 days 17:55:04
182     15 days 10:58:13
186      0 days 18:56:06
198      5 days 09:16:00
              ...       
61241    7 days 00:09:01
61242    3 days 16:52:47
61243    2 days 08:05:53
61253    4 days 07:58:25
61254    5 days 15:40:43
61256    2 days 17:27:07
61264    2 days 15:29:28
61265    3 days 13:41:53
61269    4 days 15:18:55


Define outpatient records. 

In [15]:
query = \
"""
select * from echodata;
"""
ed = pd.read_sql_query(query, con)
ed.head()

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality
0,59653,31038,174978.0,2102-06-14,2102-06-14 13:30:00,Endocarditis.,74.0,165.0,2.00,102/72,102.0,72.0,86.0,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
1,59654,70150,156140.0,2191-05-10,2191-05-10 15:08:00,"CHF, A-FIB",66.0,263.0,2.25,107/81,107.0,81.0,95.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal
2,59655,54190,188571.0,2158-02-16,2158-02-16 13:48:00,Cerebrovascular event/TIA. Streptococcal bacte...,72.0,187.0,2.07,147/87,147.0,87.0,90.0,Inpatient,Portable TEE (Complete),Full Doppler and color Doppler,None,Adequate
3,59656,5771,185291.0,2173-09-02,2173-09-02 09:47:00,78 year-old man with atrial fibrillation off a...,66.0,130.0,1.67,163/63,163.0,63.0,80.0,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
4,59657,80030,100442.0,2119-06-07,2119-06-07 14:59:00,Left ventricular function.,63.0,126.0,1.59,88/53,88.0,53.0,115.0,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate


In [141]:
ed['weight'].isnull().value_counts()

False    38957
True      6837
Name: weight, dtype: int64

In [19]:
(ed['status'] == 'Outpatient').value_counts()

False    39075
True      6719
Name: status, dtype: int64

In [22]:
(~ed['hadm_id'].isnull()).value_counts()

True     34037
False    11757
Name: hadm_id, dtype: int64

People marked as outpatient who have hospital admission ids? 

In [39]:
ed.loc[(ed['status'] == 'Outpatient') & (~ed['hadm_id'].isnull())].head()

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality
1,59654,70150,156140.0,2191-05-10,2191-05-10 15:08:00,"CHF, A-FIB",66.0,263.0,2.25,107/81,107.0,81.0,95.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal
26,59812,82559,197325.0,2157-02-08,2157-02-08 09:41:00,Endocarditis.,60.0,108.0,1.44,153/68,153.0,68.0,59.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Adequate
32,59818,22180,186367.0,2134-08-13,2134-08-13 15:06:00,Hypertension. Shortness of breath. Resp failure.,NaN,174.0,NaN,115/41,115.0,41.0,65.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal
166,60363,69811,142089.0,2170-08-06,2170-08-06 11:16:00,Left ventricular function.,NaN,150.0,NaN,114/76,114.0,76.0,82.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal
202,59966,55357,151470.0,2198-04-17,2198-04-17 10:00:00,f/u MR,NaN,125.0,1.64,143/89,143.0,89.0,101.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Adequate


People who have no hospital admission id but who are marked as inpatient? 

In [28]:
ed.loc[(ed['status'] == 'Inpatient') & (ed['hadm_id'].isnull())].head()

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality
7,59660,82208,NaN,2200-01-02,2200-01-02 11:23:00,Congestive heart failure. Hypertension. Shortn...,59.0,143.0,1.60,170/82,170.0,82.0,92.0,Inpatient,TTE (Complete),Full Doppler and color Doppler,None,Adequate
8,59669,15472,NaN,2176-05-28,2176-05-28 08:58:00,Left ventricular function. Shortness of breath...,72.0,200.0,2.13,122/90,122.0,90.0,NaN,Inpatient,TTE(Focused views),Focused pulse and color flow,None,Suboptimal
21,59807,98046,NaN,2198-09-13,2198-09-13 10:26:00,"s/p AVR, CABG. Here with CHF exacerbation.",73.0,217.0,2.23,117/86,117.0,86.0,113.0,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Suboptimal
29,59815,22180,NaN,2136-05-22,2136-05-22 14:53:00,Left ventricular function. Right ventricular f...,66.0,190.0,1.96,150/62,150.0,62.0,96.0,Inpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal
30,59816,22180,NaN,2136-03-26,2136-03-26 08:31:00,Left ventricular function. Right ventricular f...,66.0,185.0,1.94,122/50,122.0,50.0,77.0,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate


Looking at a patient who was marked as outpatient who has a hospital admission id

In [49]:
query = \
"""
select * from admissions where subject_id = 70150;
"""
am = pd.read_sql_query(query, con)
am.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,49507,70150,156140,2191-05-09 15:17:00,2191-05-14 19:18:00,None,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,PROTESTANT QUAKER,MARRIED,BLACK/AFRICAN AMERICAN,2191-05-09 13:19:00,2191-05-09 16:36:00,CONGESTIVE HEART FAILURE,0,1


In [50]:
query = \
"""
select * from noteevents where subject_id = 70150 and category = 'Echo';
"""
ne = pd.read_sql_query(query, con)
ne.head()

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,59654,70150,156140,2191-05-10,None,None,Echo,Report,None,None,"PATIENT/TEST INFORMATION:\nIndication: CHF, A-..."


In [51]:
query = \
"""
select * from echodata where subject_id = 70150;
"""
ed_ = pd.read_sql_query(query, con)
ed_.head()

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality
0,59654,70150,156140,2191-05-10,2191-05-10 15:08:00,"CHF, A-FIB",66.0,263.0,2.25,107/81,107.0,81.0,95.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal


Looking at a patient who has no hospital admission id but who was marked as inpatient:

In [33]:
query = \
"""
select * from admissions where subject_id = 82208;
"""
am = pd.read_sql_query(query, con)
am.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,53371,82208,144407,2199-03-18 18:29:00,2199-03-25 16:50:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,CATHOLIC,DIVORCED,WHITE,2199-03-18 13:22:00,2199-03-18 21:23:00,HTN EMERGENCY,0,1
1,53372,82208,139033,2200-04-25 07:15:00,2200-04-28 17:00:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicaid,ENGL,CATHOLIC,DIVORCED,WHITE,NaT,NaT,LEFT RENAL TUMOR/SDA,0,1
2,53373,82208,188268,2200-10-21 14:54:00,2200-10-27 21:29:00,2200-10-27 21:29:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicaid,ENGL,CATHOLIC,DIVORCED,WHITE,2200-10-21 11:38:00,2200-10-21 16:09:00,RESPIRATORY DISTRESS,1,1


In [43]:
query = \
"""
select * from noteevents where subject_id = 82208 and category = 'Echo';
"""
ne = pd.read_sql_query(query, con)
ne.head()

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,59659,82208,188268.0,2200-10-22,None,None,Echo,Report,None,None,PATIENT/TEST INFORMATION:\nIndication: Acute M...
1,59660,82208,NaN,2200-01-02,None,None,Echo,Report,None,None,PATIENT/TEST INFORMATION:\nIndication: Congest...


In [41]:
query = \
"""
select * from echodata where subject_id = 82208;
"""
ed_ = pd.read_sql_query(query, con)
ed_.head()

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality
0,59659,82208,188268.0,2200-10-22,2200-10-22 15:14:00,"Acute MR worsening,ESRD, on HD, NSLS lung Ca, ...",59.0,136.0,1.57,126/61,126.0,61.0,95.0,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
1,59660,82208,NaN,2200-01-02,2200-01-02 11:23:00,Congestive heart failure. Hypertension. Shortn...,59.0,143.0,1.60,170/82,170.0,82.0,92.0,Inpatient,TTE (Complete),Full Doppler and color Doppler,None,Adequate


Defining outpatient echos as echos with no hospital admission id or marked as outpatient. 

In [110]:
query = \
"""
with outpatient_echos as (
    select * from echodata 
    where hadm_id is null
    or status = 'Outpatient'
)
, echo_annotations_qual AS (
    SELECT *
        , (CASE WHEN tv_pulm_htn        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN tv_regurgitation   IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN tv_stenosis        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_cavity          IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_diastolic       IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_systolic        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_wall            IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_cavity          IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_volume_overload IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_systolic        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_wall            IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN av_regurgitation   IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN av_stenosis        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN mv_regurgitation   IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN mv_stenosis        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN la_cavity          IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN ra_dilated         IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN ra_pressure        IS NULL THEN 1 ELSE 0 END) AS num_not_null
    FROM echo_annotations_unique
)
, echo_annotations_unique_ AS (
    SELECT DISTINCT ON (hadm_id, new_time) *
    FROM echo_annotations_qual
    ORDER BY hadm_id, new_time, num_not_null DESC
)
select op.*, ea.key
from outpatient_echos op 
left join echo_annotations_unique_ ea
    on op.hadm_id = ea.hadm_id and
    op.charttime is not distinct from ea.new_time
"""
foo = pd.read_sql_query(query, con)
foo.head()

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality,key
0,71919,58947,100037.0,2183-04-15,2183-04-15 08:19:00,Endocarditis. Left ventricular function. Right...,71.0,220.0,2.20,120/67,120.0,67.0,76.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Adequate,67444.0
1,95017,53014,100039.0,2174-05-02,2174-05-02 15:44:00,Cardiomyopathy,64.0,145.0,1.71,120/85,120.0,85.0,85.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Adequate,90542.0
2,84885,91327,100465.0,2111-05-08,2111-05-08 11:46:00,Pericardial effusion.,65.0,160.0,1.80,104/64,104.0,64.0,NaN,Outpatient,TTE (Focused views),Limited Doppler and no color Doppler,None,Adequate,80410.0
3,62209,18982,100522.0,2139-07-22,2139-07-22 15:50:00,Myocardial infarction. Left ventricular funct...,63.0,195.0,1.91,106/87,106.0,87.0,89.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal,57734.0
4,97621,720,100753.0,2160-05-06,2160-05-06 14:52:00,Left ventricular function.,68.0,191.0,2.01,104/57,104.0,57.0,99.0,Outpatient,TTE (Complete),Full doppler and color doppler,None,Adequate,93146.0


In [114]:
foo['key'].isnull().value_counts()

True     11757
False     1078
Name: key, dtype: int64

Number of patients with outpatient echos AFTER.

In [118]:
query = \
"""
with outpatient_echos as (
select * from echodata 
where hadm_id is null
or status = 'Outpatient'
)
, echo_annotations_qual AS (
    SELECT *
        , (CASE WHEN tv_pulm_htn        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN tv_regurgitation   IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN tv_stenosis        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_cavity          IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_diastolic       IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_systolic        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_wall            IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_cavity          IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_volume_overload IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_systolic        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_wall            IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN av_regurgitation   IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN av_stenosis        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN mv_regurgitation   IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN mv_stenosis        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN la_cavity          IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN ra_dilated         IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN ra_pressure        IS NULL THEN 1 ELSE 0 END) AS num_not_null
    FROM echo_annotations_unique
)
, echo_annotations_unique_ AS (
    SELECT DISTINCT ON (hadm_id, new_time) *
    FROM echo_annotations_qual
    ORDER BY hadm_id, new_time, num_not_null DESC
)
select ifs.subject_id
    ,ifs.icustay_id as ip_charttime
    ,ifs.hadm_id as ip_hadm_id
    ,ifs.ed_charttime as ip_charttime
    --,ifs.ea_lv_systolic as ip_lv_systolic
    ,ea_key as ip_key
    ,op.hadm_id as op_hadm_id
    ,op.charttime as op_charttime
    ,op.status as op_status
    --,ea.lv_systolic as ea_lv_systolic
    ,ea.key as op_key
from icu_features ifs
inner join outpatient_echos op
    on ifs.subject_id = op.subject_id
left join echo_annotations_unique_ ea
    on op.hadm_id = ea.hadm_id and
    op.charttime is not distinct from ea.new_time
where use_record = 1
and ifs.ed_charttime < op.charttime
"""
foo = pd.read_sql_query(query, con)
foo.sort_values('op_charttime', ascending = False).groupby('subject_id').first().head(n = 10)

,ip_charttime,ip_hadm_id,ip_charttime,ip_key,op_hadm_id,op_charttime,op_status,op_key
subject_id,,,,,,,,
52,261857,190797,2191-01-10 11:21:00,78325,NaN,2191-08-03 16:17:00,Inpatient,NaN
145,226841,198161,2144-03-29 14:46:00,74511,NaN,2145-07-08 11:36:00,Inpatient,NaN
156,260696,168847,2120-06-03 12:52:00,60695,NaN,2120-10-21 13:45:00,Outpatient,NaN
368,242607,105889,2137-07-12 13:26:00,69134,NaN,2139-11-16 10:43:00,Inpatient,NaN
402,203761,177951,2155-05-23 11:15:00,67811,NaN,2156-09-06 13:00:00,Outpatient,NaN
720,243412,100753,2160-05-06 14:52:00,93146,100753.0,2160-05-14 14:41:00,Outpatient,93115.0
745,244976,162004,2202-03-23 10:55:00,59143,NaN,2203-03-24 15:05:00,Inpatient,NaN
801,201545,195211,2197-08-22 12:34:00,95147,NaN,2198-04-04 09:50:00,Inpatient,NaN
820,219249,180654,2142-08-16 16:10:00,61696,NaN,2143-12-12 11:33:00,Inpatient,NaN


In [119]:
foo['op_key'].isnull().value_counts()

True     811
False     79
Name: op_key, dtype: int64

Number of patients with outpatient echos BEFORE.

In [120]:
query = \
"""
with outpatient_echos as (
select * from echodata 
where hadm_id is null
or status = 'Outpatient'
)
, echo_annotations_qual AS (
    SELECT *
        , (CASE WHEN tv_pulm_htn        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN tv_regurgitation   IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN tv_stenosis        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_cavity          IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_diastolic       IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_systolic        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN lv_wall            IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_cavity          IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_volume_overload IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_systolic        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN rv_wall            IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN av_regurgitation   IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN av_stenosis        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN mv_regurgitation   IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN mv_stenosis        IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN la_cavity          IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN ra_dilated         IS NULL THEN 1 ELSE 0 END) +
          (CASE WHEN ra_pressure        IS NULL THEN 1 ELSE 0 END) AS num_not_null
    FROM echo_annotations_unique
)
, echo_annotations_unique_ AS (
    SELECT DISTINCT ON (hadm_id, new_time) *
    FROM echo_annotations_qual
    ORDER BY hadm_id, new_time, num_not_null DESC
)
select ifs.subject_id
    ,ifs.icustay_id as ip_charttime
    ,ifs.hadm_id as ip_hadm_id
    ,ifs.ed_charttime as ip_charttime
    --,ifs.ea_lv_systolic as ip_lv_systolic
    ,ea_key as ip_key
    ,op.hadm_id as op_hadm_id
    ,op.charttime as op_charttime
    ,op.status as op_status
    --,ea.lv_systolic as ea_lv_systolic
    ,ea.key as op_key
from icu_features ifs
inner join outpatient_echos op
    on ifs.subject_id = op.subject_id
left join echo_annotations_unique_ ea
    on op.hadm_id = ea.hadm_id and
    op.charttime is not distinct from ea.new_time
where use_record = 1
and ifs.ed_charttime > op.charttime
"""
foo = pd.read_sql_query(query, con)
foo.sort_values('op_charttime', ascending = False).groupby('subject_id').first().head(n = 10)

,ip_charttime,ip_hadm_id,ip_charttime,ip_key,op_hadm_id,op_charttime,op_status,op_key
subject_id,,,,,,,,
117,217966,164853,2133-11-20 14:16:00,70595,NaN,2133-03-31 12:58:00,Inpatient,NaN
124,256064,138376,2166-01-17 11:52:00,56310,NaN,2164-11-29 17:30:00,Outpatient,NaN
307,248439,161712,2164-01-02 16:37:00,77760,161712.0,2163-12-29 10:15:00,Outpatient,77761.0
402,203761,177951,2155-05-23 11:15:00,67811,NaN,2155-04-09 11:00:00,Outpatient,NaN
720,243412,100753,2160-05-06 14:52:00,93146,171046.0,2160-02-25 11:42:00,Outpatient,93086.0
745,244976,162004,2202-03-23 10:55:00,59143,NaN,2200-03-26 11:28:00,Inpatient,NaN
773,213086,170261,2109-03-08 12:04:00,61491,NaN,2107-09-22 16:30:00,Inpatient,NaN
1006,209730,189081,2159-05-20 15:09:00,60408,NaN,2159-03-19 10:00:00,Outpatient,NaN
1018,290398,135732,2110-09-10 13:00:00,91703,NaN,2106-12-22 13:50:00,Inpatient,NaN


In [121]:
foo['op_key'].isnull().value_counts()

True     736
False     64
Name: op_key, dtype: int64